In [1]:
import torch
import pyro
from pyro.optim import Adam
import congas as cg
from congas.models import MixtureCategorical
from pyro.infer import TraceEnum_ELBO


In [2]:
data = cg.simulation_data

interface = cg.Interface()

In [3]:
data["norm_factor"] = torch.sum(data["data"],axis=0) / 10**3

In [4]:
torch.mean((data["data"] / data["norm_factor"]) / 2 )

tensor(15.6250)

In [5]:
interface.set_model(MixtureCategorical)
interface.set_optimizer(Adam)
interface.set_loss(TraceEnum_ELBO)
interface.initialize_model(data)
interface.set_model_params({"lr" : 0.5,'K': 2, "nb_size_init" : torch.ones(32) * 300,
                            "theta_scale" : torch.ones(32) * 1.5,
                            "theta_rate" : torch.ones(32) * 0.1,
                           "probs" : torch.tensor([0.2,0.6,0.2,0.05,0.05])})

In [6]:
ll = interface.run(steps = 200, MAP = True)



Running MixtureCategorical on 1000 cells wiht 32 segments for 200 steps

Done!


ELBO: -5833172.480000000  : 100%|██████████| 200/200 [01:29<00:00,  2.24it/s]


In [7]:
import numpy as np

lr = interface.learned_parameters()
lr

Computing assignment probabilities


{'mixture_weights': array([0.00374981, 0.9962502 ], dtype=float32),
 'NB_size': array([ 854.645   , 2882.4077  ,  529.0988  , 1248.5513  ,  110.25468 ,
         739.49176 , 1961.6394  ,    8.88016 ,  667.11163 , 2515.6064  ,
         661.36774 ,   11.325618, 1024.4348  , 1600.1339  ,  680.1235  ,
         403.96472 ,  743.3631  ,  395.17523 ,  753.58057 , 1445.3713  ,
        1531.5133  , 1874.4006  , 2861.2358  ,  112.5937  ,  102.89825 ,
        1623.1536  , 2252.765   , 1353.7036  ,  404.42322 ,  809.7347  ,
         127.76866 ,  436.40338 ], dtype=float32),
 'segment_factor': array([ 9.557074 , 13.469914 ,  8.332872 , 10.2422   ,  6.348126 ,
         9.4869375, 22.844927 ,  8.474821 ,  9.473433 , 18.80574  ,
         8.899551 , 14.990898 ,  9.485658 , 11.8296795,  9.740601 ,
         7.0665975, 10.284135 ,  7.1052775,  9.501918 , 23.009363 ,
        11.819632 , 13.422286 , 12.627563 ,  6.3217354,  6.2943044,
        11.823422 , 12.616173 , 10.308484 ,  7.135077 ,  8.684508 ,
      

In [8]:
import numpy as np
np.argmax(lr["assignment_probs"], axis = 0)

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [9]:
print(np.argmax(lr["CNV_probabilities"][0], axis = 1) + 1)
print(np.argmax(lr["CNV_probabilities"][1], axis = 1) + 1)


[3 3 2 3 3 3 3 3 3 3 2 3 3 3 4 3 3 3 3 4 2 3 3 3 3 3 3 3 3 3 2 3]
[3 3 2 3 3 3 3 2 3 3 2 2 3 3 4 3 3 3 3 4 2 3 3 3 3 3 3 3 3 3 2 3]


In [23]:
data["pld"]

tensor([2.0000, 2.0000, 1.0000, 2.0000, 2.0000, 2.0000, 2.0000, 1.3000, 2.0000,
        2.0000, 1.0000, 1.7000, 2.0000, 2.0000, 3.0000, 2.0000, 2.0000, 2.0000,
        2.0000, 3.0000, 1.0000, 2.0000, 2.0000, 2.0000, 2.0000, 2.0000, 2.0000,
        2.0000, 2.0000, 2.0000, 1.0000, 2.0000])